In [ ]:
#Load packages
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.svm import SVC
import re

In [ ]:
#Load dataset
df = pd.read_csv("/kaggle/input/sentiment-analysis-in-commodity-market-gold/gold-dataset-sinha-khandait.csv")

## The Price Sentiment column contains 4 classes (positive, neutral, negative and none)
* Positive denotes upward movement in price
* Negative denotes downward movement in price
* Neutral denotes sideways (steady) movement in price
* None denotes that no assessment about prices can be made from the news headline

### Let us ignore the news headlines that do not have any price movement information in it, i.e. drop rows with "Price Sentiment" as 'none'

In [ ]:
df = df[df["Price Sentiment"] != 'none']

In [ ]:
print("Commodity News Headlines")
display(df[["News","Price Sentiment"]])

### The following piece of code is used to clean the headlines

In [ ]:
def cleaner(impure_data):
    temp_list = []
    for item in impure_data:
        #finding words which start with @
        item = re.sub('@\S+', '', item)
        
        #finding words which start with http
        item = re.sub('http\S+\s*', '', item)
        
        #finding special characters, but not "emoji"
        item = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', item)
        temp_list.append(item)
    return temp_list

### Let us create a simple SVM model with tfidf vectorizer

In [ ]:
def headline_sentiment(df):
    headlines = df["News"]
    polarity = df["Price Sentiment"].tolist()
    
    #cleaning headlines i.e. removing @mentions, http(s) links and special characters such as punctuations
    clean_headline = cleaner(headlines)
    
    #initializing tf-idf vectorizer
    tf_idfvectorizer = TfidfVectorizer(sublinear_tf=True, use_idf=True)
    
    #splitting the data into train and test dataset in 70 : 30 ratio at random
    X_train, X_test, Y_train, Y_test = train_test_split(clean_headline, polarity, test_size = 0.3)
    
    train_corpus_tf_idf = tf_idfvectorizer.fit_transform(X_train) 
    test_corpus_tf_idf = tf_idfvectorizer.transform(X_test)
    
    #using SVC package to initialize a classifier with Linear kernel and other default parameters
    SVM_L = SVC(kernel= 'linear')
    
    #fitting the sparse matrix in the classifier with their respective sentiments
    SVM_L.fit(train_corpus_tf_idf, Y_train)
    
    #predicting the sentiments for the test dataset
    Y_pred = SVM_L.predict(test_corpus_tf_idf)
    
    #this prints accuracy score for the test dataset
    print("Testing Accuracy:",accuracy_score(Y_test,Y_pred))
    
    #this prints confusion matrix for the test dataset
    labels = np.unique(Y_test)    
    m = confusion_matrix(Y_test,Y_pred, labels=labels)
    print("\nConfusion matrix on test data")
    cm = pd.DataFrame(m, index=labels, columns=labels)
    cm.index = "Actual: " + cm.index
    cm.columns = "Predicted: " + cm.columns
    display(cm)
    
    #saving the data into a csv file in the current folder
    temp_df = pd.DataFrame()
    temp_df["News"] = X_test
    temp_df["Actual Price Sentiment"] = Y_test
    temp_df["Predicted Sentiment"] = Y_pred
    temp_df.to_csv("predicted.csv")
    print('Predictions on Test Data are as follows:')
    display(temp_df)

    return(tf_idfvectorizer,SVM_L)

In [ ]:
vectorizer,model = headline_sentiment(df)

### Important
* Looking at the confusion matrix, it  is clear that the performance on neutral will be poor. 
* Positive and negative headlines are likely to be identified correctly

In [ ]:
#Trying sample headlines
vector = vectorizer.transform(["Gold expected to beat expectations."])
sentiment = model.predict(vector)
print(sentiment)

In [ ]:
#Trying sample headlines
vector = vectorizer.transform(["The price of gold continues declining."])
sentiment = model.predict(vector)
print(sentiment)

In [ ]:
#Trying sample headlines
vector = vectorizer.transform(["Gold price continues to improve."])
sentiment = model.predict(vector)
print(sentiment)

In [ ]:
#Trying sample headlines
vector = vectorizer.transform(["Gold price expected to remain steady."])
sentiment = model.predict(vector)
print(sentiment)

### Try sentence transformers to get extraordinary improvement in results